In [2]:
from filterpy.common import kinematic_kf
import numpy as np
from filterpy.kalman import IMMEstimator
import math
from filterpy.kalman import KalmanFilter

In [3]:
def state_transition_matrix(v, yaw, T):
    term13 = (v/yaw) * (-math.cos(yaw) + math.cos(T * yaw + yaw))
    term14 = (-math.sin(yaw) + math.sin(T * yaw + yaw))/yaw
    term15 = T * v * math.cos(T * yaw + yaw)/ yaw - v * (-math.sin(yaw) + math.sin(T * yaw + yaw)) / yaw **2
    term23 = (v/yaw) * (-math.sin(yaw) + math.sin(T * yaw + yaw))
    term24 = (math.cos(yaw) - math.cos(T * yaw + yaw)) / yaw
    term25 = T * v * math.sin(T * yaw + yaw)/ yaw - v * (math.cos(yaw) - math.cos(T * yaw + yaw)) / yaw **2
    state_transition_matrix = np.array([[1, 0, term13, term14, term15],
                                        [0, 1, term23, term24, term25],
                                        [0, 0, 1, 0, T],
                                        [0, 0, 0, 1, 0],
                                        [0, 0, 0, 0, 1]])
    return state_transition_matrix

In [4]:
def direction(current_state, destination_state):
    directions1 = np.linalg.norm((np.array(destination_state) - np.array(current_state)))
    print(directions1)
    return directions1

In [5]:
kf1 = KalmanFilter(dim_x=5, dim_z=2)
kf2 = KalmanFilter(dim_x=5, dim_z=2)
kf3 = KalmanFilter(dim_x=5, dim_z=2)
# do some settings of x, R, P etc. here, I'll just use the defaults

In [6]:
kf1.x.shape

(5, 1)

In [7]:
start = [2, 10.3]
end_straight = [3, 0.8]
end_left = [-2.8, 5.5]
end_right = [8.3, 5.5]
v = 0.707
delta_t = 0.1

In [8]:
directions1 = direction(start, end_straight)
directions2 = direction(start, end_left)
directions3 = direction(start, end_right)

9.5524865872714
6.788225099390857
7.920227269466452


In [9]:
state_transition1 = state_transition_matrix(v, directions1, delta_t)
state_transition2 = state_transition_matrix(v, directions2, delta_t)
state_transition3 = state_transition_matrix(v, directions3, delta_t)

In [47]:
kf1.F = state_transition1
kf2.F = state_transition2
kf3.F = state_transition3

In [48]:
# define initial state [x, y, yaw, velocity, yaw*dt]
kf1.x = np.array([2.0, 10.3, directions1, v, directions1*delta_t])
kf2.x = np.array([2.0, 10.3, directions2, v, directions2*delta_t])
kf3.x = np.array([2.0, 10.3, directions3, v, directions3*delta_t])

In [63]:
sGPS = 0.5*8.8*delta_t**2  # assume 8.8m/s2 as maximum acceleration, forcing the vehicle
sCourse = 0.1*delta_t # assume 0.1rad/s as maximum turn rate for the vehicle
sVelocity = 8.8*delta_t # assume 8.8m/s2 as maximum acceleration, forcing the vehicle
sYaw = 1.0*delta_t # assume 1.0rad/s2 as the maximum turn rate acceleration for the vehicle
temp_q = np.diag([sGPS**2, sGPS**2, sCourse**2, sVelocity**2, sYaw**2])

In [64]:
temp_q.shape

(5, 5)

In [65]:
# process noise
# from filterpy.common import Q_discrete_white_noise
kf1.Q = temp_q
kf2.Q = temp_q
kf3.Q = temp_q

In [66]:
# no prediction error in second filter
# kf3.Q *= 0

In [67]:
# measurement noise
kf1.R = np.identity(2)*5
kf2.R = np.identity(2)*5
kf3.R = np.identity(2)*5

In [68]:
# Define the covariance matrix.
# Here I take advantage of the fact that P already contains np.eye(dim_x), and just multiply by the uncertainty:
kf1.P *= 10
kf2.P *= 10
kf3.P *= 10

In [69]:
kf1.Q.shape

(5, 5)

In [77]:
# define measurement function
kf1.H = np.array([[1.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 1.0, 0.0, 0.0, 0.0]])
kf2.H = np.array([[1.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 1.0, 0.0, 0.0, 0.0]])
kf3.H = np.array([[1.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 1.0, 0.0, 0.0, 0.0]])

In [23]:
print(kf1.x.shape)

(15, 1)


In [71]:
filters = [kf1, kf2, kf3]

In [72]:
mu = [0.333, 0.333, 0.333] # each filter is equally likely at the start

In [73]:
trans = np.array([[0.90, 0.05, 0.05], 
                  [0.05, 0.90, 0.05], 
                  [0.05, 0.05, 0.90]
                 ]) 
# M[i,j] is the probability of switching from filter j to filter i.

In [74]:
imm = IMMEstimator(filters, mu, trans)

In [79]:
# put some initial input here. Like the trajectory of first one second
for i in range(20):
    x = i + np.random.randn()*np.sqrt(kf1.R[0, 0])
    y = i + np.random.randn()*np.sqrt(kf1.R[1, 1])
    z = np.array([[x], [y]])
    # print(z.shape)
    
    imm.predict()
    imm.update(z)
    # print(imm.x.T)
    print("mode: ", imm.mu)

mode:  [9.98509524e-01 1.48379055e-03 6.68512367e-06]
mode:  [9.99996699e-01 6.91879279e-07 2.60884107e-06]
mode:  [9.81347702e-01 6.70355087e-04 1.79819430e-02]
mode:  [0.83570731 0.15171435 0.01257834]
mode:  [0.34175365 0.59661396 0.06163239]
mode:  [0.00950781 0.35687058 0.63362161]
mode:  [0.00162298 0.99074354 0.00763348]
mode:  [0.00183455 0.99329906 0.00486639]
mode:  [0.32970007 0.66351512 0.00678481]
mode:  [0.90892945 0.08117051 0.00990004]
mode:  [0.02619248 0.87528896 0.09851856]
mode:  [0.07810061 0.80522274 0.11667665]
mode:  [0.00772718 0.95322733 0.03904548]
mode:  [0.76867259 0.20452479 0.02680262]
mode:  [0.03076932 0.85394446 0.11528622]
mode:  [0.03291773 0.8911522  0.07593008]
mode:  [0.17889713 0.71216997 0.1089329 ]
mode:  [0.46787864 0.46852035 0.06360101]
mode:  [0.96706137 0.01072244 0.02221619]
mode:  [0.03705453 0.8508869  0.11205858]


In [51]:
kf2.F

array([[1. , 1. , 0.5, 0. , 0. , 0. ],
       [0. , 1. , 1. , 0. , 0. , 0. ],
       [0. , 0. , 1. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 1. , 1. , 0.5],
       [0. , 0. , 0. , 0. , 1. , 1. ],
       [0. , 0. , 0. , 0. , 0. , 1. ]])

In [52]:
kf1.F

array([[1. , 1. , 0.5, 0. , 0. , 0. ],
       [0. , 1. , 1. , 0. , 0. , 0. ],
       [0. , 0. , 1. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 1. , 1. , 0.5],
       [0. , 0. , 0. , 0. , 1. , 1. ],
       [0. , 0. , 0. , 0. , 0. , 1. ]])